In [ ]:
!pip install -U sagemaker --quiet # Ensure latest version of SageMaker is installed

In [13]:
print(sagemaker.__version__)

2.127.0


In [14]:
import boto3
import sagemaker
import time

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()
s3_client = boto3.client("sagemaker")

model_package_group_name = "deepfake-detection-models"  
pipeline_name = "Deepfake-detection-pipeline"  

## Get the pipeline instance

In [15]:
from pipelines.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name
)

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:258: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Submit the pipeline to SageMaker and start execution

In [16]:
pipeline.upsert(role_arn=role)


Job Name:  Deepfake/deepfake-processing-2023-01-06-23-37-32-162
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-674518009863/Deepfake-detection-pipeline/code/afb25dbff21c993917c172c8b2518724/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-674518009863/Deepfake-detection-pipeline/code/aba9cd410894d4dc1aa101779eb7c8dd/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train', 'AppManaged': False, 'S3Output': {'S3Uri': Join(on='/', values=['s3://sagemaker-us-east-1-674518009863', 'dev_datasets', 'preprocessed_data', '1673048247', 'train'])

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:674518009863:pipeline/deepfake-detection-pipeline',
 'ResponseMetadata': {'RequestId': 'dc1e6339-3953-49b9-8497-b0255e4b4cd7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc1e6339-3953-49b9-8497-b0255e4b4cd7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Fri, 06 Jan 2023 23:37:32 GMT'},
  'RetryAttempts': 0}}

In [17]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

In [ ]:
execution.describe()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

## Clean up (optional)
Delete the model registry and the pipeline to keep the studio environment tidy.

In [9]:
def delete_model_package_group(sm_client, package_group_name):
    try:
        model_versions = sm_client.list_model_packages(ModelPackageGroupName=package_group_name)

    except Exception as e:
        print("{} \n".format(e))
        return

    for model_version in model_versions["ModelPackageSummaryList"]:
        try:
            sm_client.delete_model_package(ModelPackageName=model_version["ModelPackageArn"])
        except Exception as e:
            print("{} \n".format(e))
        time.sleep(0.5)  # Ensure requests aren't throttled

    try:
        sm_client.delete_model_package_group(ModelPackageGroupName=package_group_name)
        print("{} model package group deleted".format(package_group_name))
    except Exception as e:
        print("{} \n".format(e))
    return


def delete_sagemaker_pipeline(sm_client, pipeline_name):
    try:
        sm_client.delete_pipeline(
            PipelineName=pipeline_name,
        )
        print("{} pipeline deleted".format(pipeline_name))
    except Exception as e:
        print("{} \n".format(e))
        return

In [ ]:
delete_model_package_group(s3_client, model_package_group_name)
delete_sagemaker_pipeline(s3_client, pipeline_name)

In [12]:
delete_model_package_group(s3_client, 'deepfake-detection-models')
delete_sagemaker_pipeline(s3_client, 'Deepfake-detection-pipeline')

deepfake-detection-models model package group deleted
Deepfake-detection-pipeline pipeline deleted
